<a href="https://colab.research.google.com/github/Arseniy-Polyakov/applied_linguistics_course/blob/main/Task_5_Emotional_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Данная работа будет состоять в сравнении качества двух моделей распознавания эмоций (CEDR и SEARA) на учебном датасете.
Также будет обучен алгоритм логистической регрессии на датасете sagteam/cedr_v1 и будет проведено сравнение качества данного алгоритма с моделями CEDR и SEARA

Импортируем необходимые модули для предобработки текста, токенизатора и моделей распознавания эмоций

In [1]:
!pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 33.0 MB/s eta 0:00:00


In [2]:
import re
import os
import nltk
import torch
import pymorphy3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from datasets import load_dataset
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

Загружаем списки стоп-слов для предобработки текстов. Списки стоп-слов были вручную проверены на наличие междометий, которые могут влиять на распознавании эмоции в тексте, междометий среди списка стоп-слов не оказалось

In [3]:
nltk.download("stopwords")
nltk.download("punkt_tab")
stop_words = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Создаем папку texts_emotions, куда помещаем размеченные по эмоциям тексты. Парсим эти файлы, определяем тексты и метки

In [4]:
files = os.listdir("/content/texts_emotions")

data = [pd.read_excel("/content/texts_emotions/" + document)["text"].tolist() for document in files]
labels_groups = [pd.read_excel("/content/texts_emotions/" + files[i])["emotion"] for i in range(len(data))]
labels = [label for corpus in labels_groups for label in corpus]

Пишем функцию препроцессинга текста

In [5]:
def preprocessing(text: str) -> str:
  """
  Функция для препроцессинга текста: удаление стоп-слов, знаков препинания и цифр;
  приведение к нижнему регистру, лемматизация
  """
  lemmatizer = pymorphy3.MorphAnalyzer()
  text_without_punct = re.sub(r"[^а-яё\s\-]", "", text.lower())
  text_cleaned = " ".join([lemmatizer.parse(token)[0].normal_form for token in text_without_punct.split() if token not in stop_words])
  return text_cleaned

Осуществляем препроцессинг для наших размеченных данных

In [6]:
texts = [text for corpus in data for text in corpus]
texts_preprocessed = [preprocessing(text) for text in texts]

Создаем объекты для модели CEDR (токенизатор и собственно предобученную модель)

In [7]:
model_CEDR = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny2-cedr-emotion-detection")
tokenizer_CEDR = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2-cedr-emotion-detection")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/117M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Токенизируем наши данные

In [8]:
inputs_CEDR = tokenizer_CEDR(texts_preprocessed, padding=True, truncation=True, return_tensors="pt", max_length=512)
inputs_CEDR

{'input_ids': tensor([[    2, 41585,  2371,  ...,     0,     0,     0],
        [    2, 32184, 24039,  ...,     0,     0,     0],
        [    2, 42086,  9886,  ...,     0,     0,     0],
        ...,
        [    2, 45161, 49049,  ...,     0,     0,     0],
        [    2, 45302,  6003,  ...,     0,     0,     0],
        [    2, 47189,  8703,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

Определяем вероятности принадлежности каждого предложения в данных к тому или иному классу (эмоции)

In [9]:
model_CEDR.eval()
with torch.no_grad():
  outputs = model_CEDR(**inputs_CEDR)
  logits = outputs.logits
  probabilities = torch.softmax(logits, dim=-1)
probabilities

tensor([[2.5080e-04, 9.9886e-01, 2.5119e-04, 2.7301e-04, 1.3219e-04, 2.2943e-04],
        [2.4231e-02, 5.9741e-01, 5.5387e-02, 1.3111e-02, 3.3293e-03, 3.0654e-01],
        [4.7580e-01, 5.0795e-01, 6.7211e-03, 2.4175e-03, 3.0547e-03, 4.0540e-03],
        ...,
        [9.9805e-01, 1.2410e-03, 1.7662e-04, 1.0377e-04, 1.1776e-04, 3.1401e-04],
        [9.9871e-01, 5.8922e-04, 1.0802e-04, 1.0281e-04, 1.9332e-04, 2.9753e-04],
        [1.7287e-03, 9.9625e-01, 5.1184e-04, 3.7830e-04, 1.8166e-04, 9.4897e-04]])

Обрабатываем полученные метки эмоций

In [10]:
labels_CEDR = ["no emotions", "ENJOYMENT", "DISTRESS", "EXCITEMENT", "FEAR", "ANGER"]
labels_CEDR_dataset = [labels_CEDR[j] for i in range(len(texts_preprocessed)) for j in range(len(probabilities.tolist()[i])) if probabilities.tolist()[i][j] == max( probabilities.tolist()[i])]
labels_CEDR_dataset

['ENJOYMENT',
 'ENJOYMENT',
 'ENJOYMENT',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'ENJOYMENT',
 'ENJOYMENT',
 'no emotions',
 'no emotions',
 'ANGER',
 'ANGER',
 'ANGER',
 'no emotions',
 'DISTRESS',
 'DISTRESS',
 'ANGER',
 'ANGER',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'ANGER',
 'DISTRESS',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'ENJOYMENT',
 'no emotions',
 'no emotions',
 'ENJOYMENT',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'ANGER',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'ANGER',
 'no emotions',
 'no emotions',
 'ANGER',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'ANGER',
 'no emotions',
 'ANGER',
 'no emotions',
 

Выводим метрики оценки модели CEDR на учебном датасете

In [75]:
report_cedr = classification_report(labels_CEDR_dataset, labels)
print(report_cedr)

              precision    recall  f1-score   support

       ANGER       0.16      0.19      0.17        31
     DISGUST       0.00      0.00      0.00         0
    DISTRESS       0.08      0.27      0.12        11
   ENJOYMENT       0.27      0.38      0.32        26
  EXCITEMENT       0.00      0.00      0.00         4
        FEAR       0.03      1.00      0.05         1
     STARTLE       0.00      0.00      0.00         0
 no emotions       0.00      0.00      0.00       189

    accuracy                           0.08       262
   macro avg       0.07      0.23      0.08       262
weighted avg       0.05      0.08      0.06       262



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

Создаем объекты для модели SEARA (токенизатор и собственно предобученную модель)

In [11]:
model_SEARA = AutoModelForSequenceClassification.from_pretrained("seara/rubert-tiny2-russian-emotion-detection-cedr")
tokenizer_SEARA = AutoTokenizer.from_pretrained("seara/rubert-tiny2-russian-emotion-detection-cedr")

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/117M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/410 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Токенизируем наши данные

In [12]:
inputs_SEARA = tokenizer_SEARA(texts_preprocessed, padding=True, truncation=True, return_tensors="pt", max_length=512)
inputs_SEARA

{'input_ids': tensor([[    2, 41585,  2371,  ...,     0,     0,     0],
        [    2, 32184, 24039,  ...,     0,     0,     0],
        [    2, 42086,  9886,  ...,     0,     0,     0],
        ...,
        [    2, 45161, 49049,  ...,     0,     0,     0],
        [    2, 45302,  6003,  ...,     0,     0,     0],
        [    2, 47189,  8703,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

Определяем вероятности принадлежности каждого предложения в данных к тому или иному классу (эмоции)

In [13]:
model_SEARA.eval()
with torch.no_grad():
  outputs = model_SEARA(**inputs_SEARA)
  logits = outputs.logits
  probabilities_SEARA = torch.softmax(logits, dim=-1)
probabilities_SEARA

tensor([[2.1930e-03, 9.9324e-01, 1.2306e-03, 1.4203e-03, 5.5140e-04, 1.3636e-03],
        [2.8862e-02, 4.7907e-01, 3.7344e-01, 1.0141e-02, 5.9177e-03, 1.0257e-01],
        [9.9611e-01, 1.9972e-03, 3.9531e-04, 6.6001e-04, 3.2769e-04, 5.1247e-04],
        ...,
        [9.4754e-01, 4.5334e-02, 1.0359e-03, 2.0353e-03, 1.1267e-03, 2.9241e-03],
        [9.9518e-01, 2.6576e-03, 2.9680e-04, 5.8069e-04, 4.8161e-04, 8.0733e-04],
        [1.9073e-03, 9.9094e-01, 1.2733e-03, 1.6805e-03, 6.3067e-04, 3.5681e-03]])

Выводим полученные метки эмоций

In [14]:
labels_SEARA = ["no emotions", "ENJOYMENT", "DISTRESS", "EXCITEMENT", "FEAR", "ANGER"]
labels_SEARA_dataset = [labels_SEARA[j] for i in range(len(texts_preprocessed)) for j in range(len(probabilities_SEARA.tolist()[i])) if probabilities_SEARA.tolist()[i][j] == max( probabilities_SEARA.tolist()[i])]

['ENJOYMENT',
 'ENJOYMENT',
 'ENJOYMENT',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'ENJOYMENT',
 'ENJOYMENT',
 'no emotions',
 'no emotions',
 'ANGER',
 'ANGER',
 'ANGER',
 'no emotions',
 'DISTRESS',
 'DISTRESS',
 'ANGER',
 'ANGER',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'ANGER',
 'DISTRESS',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'ENJOYMENT',
 'no emotions',
 'no emotions',
 'ENJOYMENT',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'ANGER',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'ANGER',
 'no emotions',
 'no emotions',
 'ANGER',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'no emotions',
 'ANGER',
 'no emotions',
 'ANGER',
 'no emotions',
 

Оцениваем качество модели SEARA на учебном датасете

In [79]:
report_seara = classification_report(labels_SEARA_dataset, labels)
print(report_seara)

              precision    recall  f1-score   support

       ANGER       0.11      0.29      0.15        14
     DISGUST       0.00      0.00      0.00         0
    DISTRESS       0.08      0.43      0.13         7
   ENJOYMENT       0.27      0.32      0.29        31
  EXCITEMENT       0.00      0.00      0.00         2
        FEAR       0.05      0.25      0.09         8
     STARTLE       0.00      0.00      0.00         0
 no emotions       0.00      0.00      0.00       200

    accuracy                           0.07       262
   macro avg       0.06      0.16      0.08       262
weighted avg       0.04      0.07      0.05       262



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

Считаем количество правильно определенных эмоций по сравнению с размеченным датасетом

In [15]:
correct_CEDR = {texts[i]: labels[i] for i in range(len(labels)) if labels[i] == labels_CEDR_dataset[i]}
correct_SEARA = {texts[i]: labels[i] for i in range(len(labels)) if labels[i] == labels_SEARA_dataset[i]}
correct_SEARA["0"] = None

In [16]:
print(len(correct_CEDR))
print(len(correct_SEARA))

20
20


Выведем правильно определенные тексты с меткой эмоции в таблице

In [17]:
df_labels = pd.DataFrame(correct_CEDR.keys(), index=[i for i in range(len(correct_CEDR))], columns=["Texts CEDR"])
df_labels["Labels CEDR"] = correct_CEDR.values()
df_labels["Texts SEARA"] = correct_SEARA.keys()
df_labels["Labels SEARA"] = correct_SEARA.values()
df_labels

,Texts CEDR,Labels CEDR,Texts SEARA,Labels SEARA
0,У меня никогда не было своей комнаты. Жили пол...,DISTRESS,У меня никогда не было своей комнаты. Жили пол...,DISTRESS
1,Живу на Кипре. Работаю на приличной должности....,DISTRESS,Живу на Кипре. Работаю на приличной должности....,DISTRESS
2,"Говорят, что одиночество затягивает. Я не вери...",DISTRESS,"Отношения с родителями замечательные, и все же...",DISTRESS
3,"Жена очень много читает, и это начинает дико р...",ANGER,"Жена очень много читает, и это начинает дико р...",ANGER
4,"Бесит мой начальник! Он постоянно ест, смотрит...",ANGER,"Бесит мой начальник! Он постоянно ест, смотрит...",ANGER
5,"Я учусь в школе-интернате, которая в первой де...",ANGER,Меня всегда бесила одна вещь — почему я должна...,ANGER
6,"Вот прям достало уже! Муж зарабатывает больше,...",ANGER,Как меня бесят эти совковые бабки-гардеробщицы...,ANGER
7,Меня всегда бесила одна вещь — почему я должна...,ANGER,"Бросил парень, родители в другом городе. У мен...",FEAR
8,Как меня бесят эти совковые бабки-гардеробщицы...,ANGER,"Перенервничал. Прихватило сердце, желудок, тря...",FEAR
9,"Перенервничал. Прихватило сердце, желудок, тря...",FEAR,Приехали в гости к родителям мужа. А там у отц...,ENJOYMENT


Вывод: на предоставленных данных модели CEDR и SEARA показали приблизительно одинаковые результаты, в первой итерации 19/19 SEARA/CEDR, во второй 19/20 SEARA/CEDR (как и по количеству правильно определенных классов, так и по меткам). Наиболее часто правильно определенные эмоции: ENJOYMENT, ANGER, DISTRESS

Проверим гипотезу, согласно которой наилучший результат для определения эмоций дает регрессия.

Обучим алгоритм логистической (полиномиальной) регрессии

Загрузим датасет sagteam/cedr, на основе которого обучались модели CEDR и SEARA

In [18]:
splits = {'train': 'enriched/train-00000-of-00001.parquet', 'test': 'enriched/test-00000-of-00001.parquet'}
dataset_for_cedr_seara = pd.read_parquet("hf://datasets/sagteam/cedr_v1/" + splits["train"])
dataset_for_cedr_seara.head()

,text,labels,source,sentences
0,Суровый гималайский медведь .,[],lj,"[[{'forma': 'Суровый', 'lemma': 'суровый'}, {'..."
1,"Так, обнаружено несколько проблем с дисплеем (...",[],lenta,"[[{'forma': 'Так', 'lemma': 'так'}, {'forma': ..."
2,У меня остается только один вопрос - является ...,[2],lj,"[[{'forma': 'У', 'lemma': 'у'}, {'forma': 'мен..."
3,Забавно как люди в возрасте удивляются входящи...,[0],twitter,"[[{'forma': 'Забавно', 'lemma': 'Забавно'}, {'..."
4,"Издание The Register отмечает, что у владельце...",[],lenta,"[[{'forma': 'Издание', 'lemma': 'издание'}, {'..."


Векторизуем тексты с помощью модели cointegrated/rubert-tiny2 (предобработка не проводилась в связи с большим объемом текстов > 70000 и нагрузкой на облачный GPU Google Colab).
Данный дистиллированный трансформер был выбран из-за своего подходящего объема для запуска на CPU

In [19]:
texts_dataset = dataset_for_cedr_seara["text"]

In [20]:
model_encoder = SentenceTransformer("cointegrated/rubert-tiny2")
embeddings_dataset = model_encoder.encode(texts_dataset)
embeddings_dataset

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

array([[-0.02134523,  0.02131438,  0.03299987, ...,  0.00149873,
         0.02310439, -0.02777565],
       [-0.03963038, -0.04482357,  0.03117575, ...,  0.06088354,
        -0.02491446, -0.05151265],
       [-0.00622218,  0.00603921,  0.05779946, ..., -0.007498  ,
        -0.00331193, -0.12514135],
       ...,
       [-0.01149452, -0.02570519,  0.02662134, ..., -0.02748095,
         0.06309391, -0.05091827],
       [ 0.03751341,  0.00753997,  0.04297598, ..., -0.02206679,
         0.07911865, -0.09522716],
       [ 0.00601427, -0.01093507,  0.02480299, ...,  0.00306611,
         0.07812951, -0.10432825]], dtype=float32)

Категоризуем метки с помощью алгоритма label encoder. Будем использовать данный алгоритм, а не one hot encoder, так как в алгоритме логистической регрессии на вход Y должен подаваться вектор, а не матрица

In [47]:
labels_dataset = dataset_for_cedr_seara["labels"]
labels_dataset_preprocessed = []
for label in labels_dataset:
  if len(label) == 0:
      labels_dataset_preprocessed.append("5")
  else:
    if label[0] == 0:
      labels_dataset_preprocessed.append("0")
    elif label[0] == 1:
      labels_dataset_preprocessed.append("1")
    elif label[0] == 2:
      labels_dataset_preprocessed.append("2")
    elif label[0] == 3:
      labels_dataset_preprocessed.append("3")
    elif label[0] == 4:
      labels_dataset_preprocessed.append("4")
len(labels_dataset_preprocessed)


7528

In [48]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels_dataset_preprocessed)

Создаем объект модели полиномиальной логистической регрессии, задаем параметры объекта

In [49]:
model_logistic_regression = LogisticRegression(multi_class="multinomial", max_iter=1000)

Делим наши данные на обучающую и тестовую выборки в соотношении 80/20

In [50]:
X_train, X_test, y_train, y_test = train_test_split(embeddings_dataset, encoded_labels, test_size=0.2)

Обучаем модель на векторизованном датасете sagteam/cedr_v1

In [ ]:
model_logistic_regression.fit(X_train, y_train)

Выводим метрики оценки качества модели

In [71]:
y_pred = model_logistic_regression.predict(X_test)
classification_report_model = classification_report(y_test, y_pred)
print(classification_report_model)

              precision    recall  f1-score   support

           0       0.77      0.79      0.78       316
           1       0.72      0.77      0.75       274
           2       0.71      0.43      0.53       117
           3       0.75      0.43      0.55       116
           4       0.41      0.08      0.14        85
           5       0.76      0.93      0.83       598

    accuracy                           0.75      1506
   macro avg       0.69      0.57      0.60      1506
weighted avg       0.73      0.75      0.72      1506



Векторизуем учебный датасет с помощью модели cointegrated/rubert-tiny2

In [53]:
embeddings_our_data = model_encoder.encode(texts_preprocessed)
embeddings_our_data

array([[ 0.06460818,  0.0111983 , -0.02380995, ...,  0.01451224,
        -0.06510374, -0.01211629],
       [ 0.11223956, -0.05235656, -0.01216037, ..., -0.06581941,
         0.0040033 , -0.01813723],
       [ 0.06845327, -0.00637502,  0.04340768, ..., -0.05609595,
        -0.01077004,  0.01765477],
       ...,
       [ 0.0344652 , -0.02160484,  0.01224226, ..., -0.04047382,
        -0.01503005, -0.00184585],
       [ 0.07855648, -0.00100238,  0.02630873, ..., -0.04049843,
        -0.04914269,  0.03086689],
       [ 0.08313233, -0.00950053,  0.07455792, ..., -0.03658319,
        -0.03234991, -0.01652064]], dtype=float32)

Определяем эмоции с помощью полиномиальной логистической регрессии на учебном датасете

In [72]:
y_pred_lg = model_logistic_regression.predict(embeddings_our_data)

Преобразуем категориальные метки в числовые значения в соответствии с разметкой датасета sagteam/cedr_v1. Эмоции, которые встречаются в данном датасете, но их нет среди учебного датасета, а именно: surprise, а также неразмеченные данные (без меток), обозначались как 5

In [81]:
labels_numbers = []
for label in labels:
  if label == "ANGER":
    labels_numbers.append(4)
  elif label == "FEAR":
    labels_numbers.append(3)
  elif label == "EXCITEMENT":
    labels_numbers.append(5)
  elif label == "DISTRESS":
    labels_numbers.append(1)
  elif label == "ENJOYMENT":
    labels_numbers.append(0)
  else:
    labels_numbers.append(5)
labels_numbers = np.array(labels_numbers)

Оценим качество логистической регресии для классификации эмоций на учебном датасете

In [82]:
report_our_data = classification_report(labels_numbers, y_pred_lg)
print(report_our_data)

              precision    recall  f1-score   support

           0       0.35      0.19      0.25        37
           1       0.22      0.05      0.09        38
           3       0.60      0.08      0.14        37
           4       1.00      0.03      0.05        38
           5       0.45      0.91      0.60       112

    accuracy                           0.44       262
   macro avg       0.52      0.25      0.23       262
weighted avg       0.50      0.44      0.33       262



В ходе использования предобученной модели CEDR на наших данных, лучше всего определяются следующие эмоции: ENJOYMENT (f1-score = 0.32), ANGER (f1-score = 0.17), DISTRESS (f1-score = 0.12)

В ходе использования предобученной модели SEARA на наших данных, лучше всего определяются следующие эмоции:
ENJOYMENT (f1-score = 0.29), ANGER (f1-score = 0.15), DISTRESS (f1-score = 0.13).

В ходе использования собственно обученного алгоритма полиномиальной логистической регрессии на датасете sagteam/cedr_v1, лучше всего определяются следующие эмоции: ENJOYMENT (f1-score = 0.25), FEAR (f1-score = 0.14).

Вывод: алгоритм логистической регрессии лучше определяет эмоцию FEAR (f1-score = 0.14, против 0.09 у модели SEARA и 0.05 у модели CEDR), а также имеет достаточно высокий показатель по эмоции ENJOYMENT, однако немного уступает по нему моделям CEDR и SEARA